## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/blcal/OneDrive/Documents/Class/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Isangel,VU,-19.5500,169.2667,72.16,86,36,13.94,scattered clouds
1,1,Nikolskoye,RU,59.7035,30.7861,58.87,73,78,8.79,broken clouds
2,2,Beringovskiy,RU,63.0500,179.3167,43.72,93,100,11.81,light rain
3,3,Kaitangata,NZ,-46.2817,169.8464,42.03,81,90,11.65,overcast clouds
4,4,Waipawa,NZ,-41.4122,175.5153,38.48,85,1,3.04,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 35
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Isangel,VU,-19.5500,169.2667,72.16,86,36,13.94,scattered clouds
1,1,Nikolskoye,RU,59.7035,30.7861,58.87,73,78,8.79,broken clouds
2,2,Beringovskiy,RU,63.0500,179.3167,43.72,93,100,11.81,light rain
3,3,Kaitangata,NZ,-46.2817,169.8464,42.03,81,90,11.65,overcast clouds
4,4,Waipawa,NZ,-41.4122,175.5153,38.48,85,1,3.04,clear sky
5,5,Butaritari,KI,3.0707,172.7902,81.45,75,77,13.38,broken clouds
6,6,Leningradskiy,RU,69.3833,178.4167,35.55,98,100,4.32,overcast clouds
7,7,Dunedin,NZ,-45.8742,170.5036,43.81,86,100,14.00,overcast clouds
8,8,Asau,RO,46.4333,26.4000,74.32,65,99,11.74,overcast clouds
9,9,Russell,US,32.3502,-85.1999,78.96,68,0,3.44,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                17
City                   17
Country                17
Lat                    17
Lng                    17
Max Temp               17
Humidity               17
Cloudiness             17
Wind Speed             17
Current Description    17
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Isangel,VU,72.16,scattered clouds,-19.5500,169.2667,
1,Nikolskoye,RU,58.87,broken clouds,59.7035,30.7861,
2,Beringovskiy,RU,43.72,light rain,63.0500,179.3167,
3,Kaitangata,NZ,42.03,overcast clouds,-46.2817,169.8464,
4,Waipawa,NZ,38.48,clear sky,-41.4122,175.5153,
5,Butaritari,KI,81.45,broken clouds,3.0707,172.7902,
6,Leningradskiy,RU,35.55,overcast clouds,69.3833,178.4167,
7,Dunedin,NZ,43.81,overcast clouds,-45.8742,170.5036,
8,Asau,RO,74.32,overcast clouds,46.4333,26.4000,
9,Russell,US,78.96,clear sky,32.3502,-85.1999,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url=base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].dropna()

0                               Tanna Lodge
1                   Tourist House - Sablino
2                                          
3                     Kaitangata Motor Camp
4                         Tora Coastal Walk
5                        Isles Sunset Lodge
6                                Gostinitsa
7               Scenic Hotel Southern Cross
8                        Casa Mika Ciobanus
9                                      Macs
10                           Zanzibar Hotel
11                               The George
12          Pacific Rendezvous Motel Resort
13                          Ludlow Farmstay
14               ASURE Alfresco Motor Lodge
15    Отке17.ру — жильё для командированных
16                         Tolaga Bay Motel
Name: Hotel Name, dtype: object

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))